In [3]:
import pandas as pd

In [4]:
number_of_modules = "most-enriched"
data_name = "rna_imputed"

In [5]:
module_to_genes = pd.read_csv(f'../modules_experiment/module_to_genes/module_to_genes_{data_name}_{str(number_of_modules)}.csv', index_col=0)
# module_to_genes

In [6]:
meth_df = pd.read_csv(f'../data/Cleveland/{data_name}.csv', index_col=0)

mice_df = pd.read_csv(f'data/mice/mice_{data_name}.csv',index_col=0).T

In [128]:
mice_genes = mice_df.index

missing_genes = []
for gene in module_to_genes["gene"]:
    if (gene.upper() not in mice_genes):
        missing_genes.append(gene)
print(f"there are {len(missing_genes)} missing genes")

miscased_genes = []
for gene in module_to_genes["gene"]:
    if (gene.upper() in mice_genes) and (gene not in mice_genes):
        miscased_genes.append(gene)
print(f"there are {len(miscased_genes)} mis-cased genes")
miscased_genes[:5]

there are 8 missing genes
there are 48 mis-cased genes


['C7orf60', 'C17orf49', 'C1orf27', 'C14orf166', 'C10orf76']

### change the mis-cased genes in mouse data

In [129]:
for gene in miscased_genes:
    mice_df.rename(index={gene.upper():  gene}, inplace=True)

### fill the other missing genes with a mean value

In [130]:
print(f"mice dataset is {mice_df.shape}")
mice_samples = mice_df.columns

rows_to_add = []
for gene in missing_genes:
    mean_value = meth_df[gene].mean()
    row_values = pd.Series({sample: mean_value for sample in mice_samples}, name=gene)
    rows_to_add.append(row_values)

# Concatenate all new rows at once
if rows_to_add:
    new_rows_df = pd.DataFrame(rows_to_add)
    mice_df = pd.concat([mice_df, new_rows_df], axis=0)

print(f"after insertions mice dataset is {mice_df.shape}")

mice dataset is (25865, 73)
after insertions mice dataset is (25873, 73)


### Remove random rows from big dataset and insert mice data in there

In [21]:
import random

df = meth_df[module_to_genes['gene']].copy()
mice_df = mice_df.T

selected_idxs = random.sample(list(df.index), k=mice_df.shape[0])

mice_df = mice_df.loc[:, ~mice_df.columns.duplicated()]

for new_row, target_idx in zip(mice_df.index, selected_idxs):
    df.loc[target_idx] = mice_df.loc[new_row].reindex(df.columns)
    df.rename(index={target_idx: new_row}, inplace=True)

df

,AAAS,AHCTF1,ARL6IP1,CDCA3,CLCC1,KPNA2,KPNB1,NDC1,NUP107,NUP133,...,MAPRE2,ZNF521,BNIP1,C19orf25,NAPG,NBAS,RAB18,RINT1,SCFD1,ZW10
id,,,,,,,,,,,,,,,,,,,,,
22RV1,6.11143,7.455620,8.874150,6.167600,5.940510,8.401310,8.986630,7.290280,6.886370,6.985480,...,4.544780,-3.822750,3.421270,4.334650,5.466170,5.377680,6.912050,5.794340,5.764940,5.405210
42MGBA,5.85038,6.749430,8.146210,6.003890,6.115310,8.913440,9.381900,6.698120,6.876690,5.533830,...,5.522610,4.928600,4.092310,4.331620,5.566780,5.596540,6.283410,4.951750,6.468360,5.646000
5637,5.57634,6.849300,8.154760,5.989940,4.769050,9.353430,9.701590,7.154050,6.651520,6.408590,...,3.742840,-3.610150,3.804790,4.894630,5.224530,5.307340,5.746000,5.469700,5.378190,5.648140
647V,5.63590,6.751960,8.444080,6.106860,5.392020,8.883900,9.218750,6.986420,6.339520,6.459090,...,3.310490,-4.748240,3.699410,4.578970,4.940200,5.351390,5.878710,4.957920,5.113950,5.643860
Mayo.PDX.Sarkaria.63,6.06618,5.570785,10.014065,5.559341,6.828109,8.228771,8.202475,6.601396,5.537845,6.560982,...,7.593002,3.437833,4.267462,5.195364,5.087961,4.964497,5.928098,5.911479,5.311003,5.360822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YD8,5.38706,6.971430,8.316730,4.978440,5.749230,7.742340,8.827310,5.997150,6.337580,6.303370,...,4.613660,-1.715350,2.911790,4.041090,5.326910,5.591670,6.977210,4.615790,6.434020,4.425960
YH13,5.56240,6.899500,8.295610,5.259900,5.916450,8.290960,9.209040,6.007520,6.301340,6.018690,...,5.471900,4.814040,3.518170,4.306470,5.418790,5.712310,6.188330,5.543100,6.323810,5.042760
YKG1,5.74626,6.907310,8.569910,5.322320,5.716350,8.665860,9.150470,5.903990,6.031470,6.131290,...,5.746600,2.558300,3.556350,4.297700,5.143550,5.724410,6.106530,5.736460,5.173250,4.803090


In [22]:
df.to_csv(f'data/prepare_mice_data_for_gsva/mice_{data_name}_{str(number_of_modules)}.csv')

## Now, run GSVA
run `calculateGSVA.R` file with `mice_data` parameter set to TRUE.

### Load the GSVA result:

In [7]:
gsva_meth = pd.read_csv(f'../modules_experiment/gsva_scores/gsva_scores_{data_name}_{number_of_modules}.csv')
gsva_mice = pd.read_csv(f'../modules_experiment/gsva_scores/mice_gsva_scores_{data_name}_{number_of_modules}.csv')


In [8]:
gsva_meth.head()

,Unnamed: 0,8,14,38,39,43,47,48,49,50,...,5126,5127,5130,5140,5143,5145,5148,5168,5190,5229
0,22RV1,0.436903,0.000555,-0.026934,-0.236241,-0.001655,0.041915,0.365020,-0.466141,0.147907,...,-0.380139,-0.074167,-0.099280,-0.049004,0.478495,-0.016778,0.445481,0.667144,-0.022923,0.222467
1,42MGBA,-0.097515,-0.062043,0.178450,0.251463,-0.020668,-0.024889,-0.158225,0.336751,0.419054,...,0.268576,-0.071268,0.252576,-0.055258,0.045041,-0.091967,0.231136,-0.055885,-0.479137,0.586704
2,5637,0.477699,-0.024133,-0.138267,0.146309,-0.043991,-0.076821,-0.456752,0.406816,0.288758,...,0.826131,0.830637,0.503483,0.179647,-0.576237,-0.334832,0.187405,-0.200009,0.189729,-0.017562
3,647V,0.171252,0.339635,0.305165,0.440819,0.177682,0.268122,0.141566,0.356177,0.167331,...,0.706533,0.650215,0.265021,0.272398,-0.639342,-0.147391,-0.217292,0.506946,-0.287527,-0.317061
4,769P,0.336379,0.402448,0.060402,0.446750,0.260591,0.173967,-0.232140,0.377659,0.161707,...,0.661838,0.718206,0.280468,-0.384513,-0.028050,-0.120623,-0.186102,-0.552712,0.082844,-0.036330


### Extract the results for mice:

In [9]:
gsva_mice_filtered = gsva_mice[gsva_mice['Unnamed: 0'].fillna('').str.startswith("Mayo")]
gsva_mice_filtered.set_index('Unnamed: 0', inplace=True)
gsva_mice_filtered

,8,14,38,39,43,47,48,49,50,51,...,5126,5127,5130,5140,5143,5145,5148,5168,5190,5229
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Mayo.PDX.Sarkaria.63,-0.336921,-0.263427,-0.366245,-0.424706,-0.296281,-0.248310,-0.431025,-0.548465,0.079907,0.385993,...,0.092801,0.402362,0.706265,0.186967,-0.285515,0.180904,-0.017188,0.535107,0.245582,-0.290200
Mayo.PDX.Sarkaria.39,-0.401568,0.176183,0.053975,0.340634,-0.024247,-0.083397,-0.371746,-0.426306,0.148168,-0.069447,...,0.490414,0.731034,-0.533923,-0.778476,-0.515611,-0.848444,-0.679008,0.036029,0.291104,0.138920
Mayo.PDX.Sarkaria.85,-0.292201,-0.307642,-0.310977,-0.392120,-0.338164,-0.363191,-0.136427,-0.580283,-0.029326,-0.158816,...,-0.119860,0.190389,0.393288,-0.570941,-0.057149,0.085884,-0.320051,0.612547,0.164663,-0.482352
Mayo.PDX.Sarkaria.206,-0.460941,0.296401,-0.215355,0.353127,0.106723,0.125406,0.020773,-0.461484,-0.156530,-0.262308,...,-0.583121,0.208195,-0.078718,-0.625327,0.166791,-0.167320,-0.692888,0.855476,0.498726,0.165928
Mayo.PDX.Sarkaria.6,-0.363285,-0.245898,-0.034055,-0.079620,-0.341865,-0.231639,-0.204886,-0.432633,0.101967,-0.041516,...,0.204096,0.320787,0.095003,-0.578438,-0.254061,-0.708549,-0.482119,0.731627,0.326167,0.178904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mayo.PDX.Sarkaria.38,-0.364884,-0.399761,-0.260435,-0.428955,-0.411705,-0.324730,-0.300264,-0.572207,0.015938,0.013830,...,-0.076828,0.087024,0.273106,-0.488289,-0.126587,0.092017,-0.625982,0.593805,-0.062548,-0.108477
Mayo.PDX.Sarkaria.148,-0.550419,0.021740,-0.067560,0.014502,-0.188049,-0.046722,0.049470,-0.527502,-0.219459,-0.269330,...,-0.289415,0.004248,-0.524889,-0.090665,-0.676052,-0.310744,-0.587435,0.709563,0.345588,0.261623
Mayo.PDX.Sarkaria.118,-0.391786,-0.124689,-0.142562,-0.133726,-0.227797,-0.262478,0.359719,-0.511181,-0.204451,-0.230687,...,0.026755,0.516028,0.167123,-0.431596,0.277677,-0.154136,-0.103155,0.737564,0.255046,0.247182


In [10]:
gsva_mice_filtered.to_csv(f'data/mice_gsva/mice_gsva_scores_{data_name}_{number_of_modules}.csv')

# (Investigate whether shuffling rows/columns changes GSVA svcores)

In [148]:
gsva = pd.read_csv(f'../modules_experiment/gsva_scores/gsva_scores_rna_imputed_500.csv', index_col=0)
gsva_genes = pd.read_csv(f'../modules_experiment/gsva_scores/gsva_scores_rna_imputed_500_genes_shuffled.csv', index_col=0)
gsva_samples = pd.read_csv(f'../modules_experiment/gsva_scores/gsva_scores_rna_imputed_500_samples_shuffled.csv', index_col=0)

gsva.sort_index().head()

,60,71,92,120,129,141,189,319,333,344,...,5110,5114,5117,5123,5125,5126,5130,5143,5145,5168
127399,0.071582,-0.478777,0.295706,0.333871,0.134090,-0.052692,-0.440097,0.146801,-0.161182,0.027259,...,-0.930000,0.005248,-0.407472,-0.382102,-0.361932,0.556420,-0.251510,-0.708709,-0.290691,0.038867
143B,0.553356,-0.537693,0.283542,-0.095425,-0.007268,-0.463580,-0.441537,-0.020686,-0.295241,-0.065583,...,-0.653125,-0.014705,-0.160089,0.433754,-0.494253,0.430698,-0.132559,-0.169352,-0.671484,-0.933926
201T,-0.262857,-0.444693,-0.007394,0.409083,0.146002,0.521969,-0.618651,0.035421,-0.335872,-0.453813,...,-0.632095,0.238702,-0.696343,-0.524273,0.010507,-0.104136,-0.071214,-0.403137,-0.489048,-0.539145
22RV1,-0.249944,0.366830,-0.094552,-0.340726,0.263741,-0.354639,-0.471379,0.184404,0.251449,0.283323,...,-0.207222,0.329217,0.321923,-0.140264,0.286906,-0.168605,-0.176099,0.277381,0.203639,0.735147
2313287,-0.357830,-0.290119,-0.347079,0.210374,0.265396,0.053786,-0.663600,-0.570916,0.047425,0.522239,...,-0.598333,0.439843,-0.038799,-0.276822,0.569512,-0.020830,0.023268,0.274429,-0.493869,-0.622500


In [149]:
print("genes shuffled")
gsva_genes.sort_index().head()

genes shuffled


,60,71,92,120,129,141,189,319,333,344,...,5110,5114,5117,5123,5125,5126,5130,5143,5145,5168
127399,0.071582,-0.478777,0.295706,0.333871,0.134090,-0.052692,-0.440097,0.146801,-0.161182,0.027259,...,-0.930000,0.005248,-0.407472,-0.382102,-0.361932,0.556420,-0.251510,-0.708709,-0.290691,0.038867
143B,0.553356,-0.537693,0.283542,-0.095425,-0.007268,-0.463580,-0.441537,-0.020686,-0.295241,-0.065583,...,-0.653125,-0.014705,-0.160089,0.433754,-0.494253,0.430698,-0.132559,-0.169352,-0.671484,-0.933926
201T,-0.262857,-0.444693,-0.007394,0.409083,0.146002,0.521969,-0.618651,0.035421,-0.335872,-0.453813,...,-0.632095,0.238702,-0.696343,-0.524273,0.010507,-0.104136,-0.071214,-0.403137,-0.489048,-0.539145
22RV1,-0.249944,0.366830,-0.094552,-0.340726,0.263741,-0.354639,-0.471379,0.184404,0.251449,0.283323,...,-0.207222,0.329217,0.321923,-0.140264,0.286906,-0.168605,-0.176099,0.277381,0.203639,0.735147
2313287,-0.357830,-0.290119,-0.347079,0.210374,0.265396,0.053786,-0.663600,-0.570916,0.047425,0.522239,...,-0.598333,0.439843,-0.038799,-0.276822,0.569512,-0.020830,0.023268,0.274429,-0.493869,-0.622500


In [150]:
print("samples shuffled")
gsva_samples.sort_index().head()

samples shuffled


,60,71,92,120,129,141,189,319,333,344,...,5110,5114,5117,5123,5125,5126,5130,5143,5145,5168
127399,0.071582,-0.478777,0.295706,0.333871,0.134090,-0.052692,-0.440097,0.146801,-0.161182,0.027259,...,-0.930000,0.005248,-0.407472,-0.382102,-0.361932,0.556420,-0.251510,-0.708709,-0.290691,0.038867
143B,0.553356,-0.537693,0.283542,-0.095425,-0.007268,-0.463580,-0.441537,-0.020686,-0.295241,-0.065583,...,-0.653125,-0.014705,-0.160089,0.433754,-0.494253,0.430698,-0.132559,-0.169352,-0.671484,-0.933926
201T,-0.262857,-0.444693,-0.007394,0.409083,0.146002,0.521969,-0.618651,0.035421,-0.335872,-0.453813,...,-0.632095,0.238702,-0.696343,-0.524273,0.010507,-0.104136,-0.071214,-0.403137,-0.489048,-0.539145
22RV1,-0.249944,0.366830,-0.094552,-0.340726,0.263741,-0.354639,-0.471379,0.184404,0.251449,0.283323,...,-0.207222,0.329217,0.321923,-0.140264,0.286906,-0.168605,-0.176099,0.277381,0.203639,0.735147
2313287,-0.357830,-0.290119,-0.347079,0.210374,0.265396,0.053786,-0.663600,-0.570916,0.047425,0.522239,...,-0.598333,0.439843,-0.038799,-0.276822,0.569512,-0.020830,0.023268,0.274429,-0.493869,-0.622500
